In [1]:
import pandas as pd

# carregar os arquivos
# 'on_bad_lines' pra pular linhas com erro se houver, e encoding utf-8
df_docvirt = pd.read_csv('dados_docvirt.csv')
df_extraidos = pd.read_csv('dados_extraidos.csv')

# ajustar 'dados_extraidos' 
# o arquivo extraído tem muitas colunas vazias no final, vamos pegar apenas as úteis
cols_uteis = ['titulo', 'conteudo']
df_extraidos = df_extraidos[cols_uteis].copy()

# adc as colunas que faltam para ficar no padrão do docvirt
# se não houver informação, deixamos como None ou um texto padrão
df_extraidos['pagina'] = None 
df_extraidos['status'] = 'extraido_novo' 

# reordenar as colunas do extraído para ficarem na mesma ordem do docvirt
colunas_ordem = df_docvirt.columns.tolist()
# garante que o df_extraidos tenha todas as colunas necessárias
for col in colunas_ordem:
    if col not in df_extraidos.columns:
        df_extraidos[col] = None
        
df_extraidos = df_extraidos[colunas_ordem]

# juntar e completar
# concatenamos os dois e colocamos o docvirt primeiro, pois ele parece mais completo 
df_final = pd.concat([df_docvirt, df_extraidos], ignore_index=True)

# remover duplicatas
# se o mesmo 'titulo' existir nos dois, mantemos o primeiro (que veio do docvirt)
# isso garante que completamos a lista sem criar linhas repetidas
df_final = df_final.drop_duplicates(subset=['titulo'], keep='first')

# visualziar e salvar
print(f"Total de linhas no DocVirt original: {len(df_docvirt)}")
print(f"Total de linhas no Extraídos original: {len(df_extraidos)}")
print(f"Total de linhas após juntar e limpar: {len(df_final)}")

# primeir slinhas
display(df_final.head())

# salva como um csv
df_final.to_csv('dados_consolidados_completo.csv', index=False)
print("Arquivo 'dados_consolidados_completo.csv' salvo com sucesso!")

Total de linhas no DocVirt original: 9126
Total de linhas no Extraídos original: 9126
Total de linhas após juntar e limpar: 5803


,pagina,titulo,conteudo,status
0,1,(CO) Consulta Correspondência\CO_0001,COD_TIPO: CO\nNUM_ANTDOC: 1\nPAGINAÇÃO: \nTIPO...,sucesso
1,2,(CO) Consulta Correspondência\CO_0002,COD_TIPO: CO\nNUM_ANTDOC: 2\nPAGINAÇÃO: \nTIPO...,sucesso
2,3,(CO) Consulta Correspondência\CO_0003,COD_TIPO: CO\nNUM_ANTDOC: 3\nPAGINAÇÃO: \nTIPO...,sucesso
3,4,(CO) Consulta Correspondência\CO_0004,COD_TIPO: CO\nNUM_ANTDOC: 4\nPAGINAÇÃO: \nTIPO...,sucesso
4,5,(CO) Consulta Correspondência\CO_0005,COD_TIPO: CO\nNUM_ANTDOC: 5\nPAGINAÇÃO: \nTIPO...,sucesso


Arquivo 'dados_consolidados_completo.csv' salvo com sucesso!


In [2]:
df_final.head(10)

,pagina,titulo,conteudo,status
0,1,(CO) Consulta Correspondência\CO_0001,COD_TIPO: CO\nNUM_ANTDOC: 1\nPAGINAÇÃO: \nTIPO...,sucesso
1,2,(CO) Consulta Correspondência\CO_0002,COD_TIPO: CO\nNUM_ANTDOC: 2\nPAGINAÇÃO: \nTIPO...,sucesso
2,3,(CO) Consulta Correspondência\CO_0003,COD_TIPO: CO\nNUM_ANTDOC: 3\nPAGINAÇÃO: \nTIPO...,sucesso
3,4,(CO) Consulta Correspondência\CO_0004,COD_TIPO: CO\nNUM_ANTDOC: 4\nPAGINAÇÃO: \nTIPO...,sucesso
4,5,(CO) Consulta Correspondência\CO_0005,COD_TIPO: CO\nNUM_ANTDOC: 5\nPAGINAÇÃO: \nTIPO...,sucesso
5,6,(CO) Consulta Correspondência\CO_0006,COD_TIPO: CO\nNUM_ANTDOC: 6\nPAGINAÇÃO: \nTIPO...,sucesso
6,7,(CO) Consulta Correspondência\CO_0007,COD_TIPO: CO\nNUM_ANTDOC: 7\nPAGINAÇÃO: \nTIPO...,sucesso
8,9,(CO) Consulta Correspondência\CO_0008,COD_TIPO: CO\nNUM_ANTDOC: 8\nPAGINAÇÃO: \nTIPO...,sucesso
9,10,(CO) Consulta Correspondência\CO_0009,COD_TIPO: CO\nNUM_ANTDOC: 9\nPAGINAÇÃO: 3 p.\n...,sucesso
12,13,(CO) Consulta Correspondência\CO_0010,COD_TIPO: CO\nNUM_ANTDOC: 10\nPAGINAÇÃO: \nTIP...,sucesso


In [3]:
# identificar onde o conteúdo está vazio ou nulo
mask_sem_conteudo = (
    df_final['conteudo'].isna() | 
    (df_final['conteudo'].astype(str).str.strip() == '') | 
    (df_final['conteudo'].astype(str).str.lower() == 'nan')
)

df_manual = df_final[mask_sem_conteudo].copy()

# remover linhas que não têm ID
df_manual = df_manual.dropna(subset=['titulo'])

# selecionar apenas as colunas úteis para sua busca manual
colunas_para_exportar = ['titulo', 'pagina', 'status']
df_manual = df_manual[colunas_para_exportar]

# criar uma coluna vazia para preencher
df_manual['conteudo_manual'] = ''

# mostra quantos sobraram
print(f"Existem {len(df_manual)} documentos reais para buscar e preencher manualmente.")
display(df_manual.head())

'''# salvar
df_manual.to_csv('para_preencher_manual.csv', index=False)
print("Arquivo 'para_preencher_manual.csv' gerado")'''

Existem 1671 documentos reais para buscar e preencher manualmente.


,titulo,pagina,status,conteudo_manual
5360,(CO) Consulta Correspondência\CO_3458,5361,vazio_ou_imagem,
5362,(CO) Consulta Correspondência\CO_3460,5363,vazio_ou_imagem,
5363,(CO) Consulta Correspondência\CO_3461,5364,vazio_ou_imagem,
5364,(CO) Consulta Correspondência\CO_3462,5365,vazio_ou_imagem,
5366,(CO) Consulta Correspondência\CO_3464,5367,vazio_ou_imagem,


'# salvar\ndf_manual.to_csv(\'para_preencher_manual.csv\', index=False)\nprint("Arquivo \'para_preencher_manual.csv\' gerado")'

In [4]:
# carregar os arquivos
# 'on_bad_lines' para pular linhas com erro se houver, e encoding utf-8
df_docvirt = pd.read_csv('dados_consolidados_completo.csv')
df_extraidos = pd.read_csv('dados_extraidos2.csv')

# ajustar 'dados_extraidos' 
# o arquivo extraído tem muitas colunas vazias no final, vamos pegar apenas as úteis
cols_uteis = ['titulo', 'conteudo']
df_extraidos = df_extraidos[cols_uteis].copy()

# adicionar as colunas que faltam para ficar no padrão do docvirt
# se não houver informação, deixamos como None (NaN) ou um texto padrão
df_extraidos['pagina'] = None 
df_extraidos['status'] = 'extraido_novo' # Marcador para você saber a origem depois

# reordenar as colunas do extraído para ficarem na mesma ordem do docvirt
colunas_ordem = df_docvirt.columns.tolist()
# garante que o df_extraidos tenha todas as colunas necessárias
for col in colunas_ordem:
    if col not in df_extraidos.columns:
        df_extraidos[col] = None
        
df_extraidos = df_extraidos[colunas_ordem]

# juntar e completar
# concatenamos os dois e colocamos o docvirt primeiro, pois ele parece mais completo (tem paginação)
df_final = pd.concat([df_docvirt, df_extraidos], ignore_index=True)

# remover duplicatas
# se o mesmo 'titulo' existir nos dois, mantemos o primeiro (que veio do docvirt)
# isso garante que completamos a lista sem criar linhas repetidas
df_final = df_final.drop_duplicates(subset=['titulo'], keep='first')

# visualziar e salvar
print(f"Total de linhas no DocVirt original: {len(df_docvirt)}")
print(f"Total de linhas no Extraídos original: {len(df_extraidos)}")
print(f"Total de linhas após juntar e limpar: {len(df_final)}")

# exibir as primeiras linhas 
display(df_final.head())

# salva
df_final.to_csv('dados_consolidados_completo2.csv', index=False)
print("Arquivo 'dados_consolidados_completo2.csv' salvo com sucesso!")

Total de linhas no DocVirt original: 5803
Total de linhas no Extraídos original: 856
Total de linhas após juntar e limpar: 5803


/tmp/ipykernel_36144/3837558105.py:27: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_final = pd.concat([df_docvirt, df_extraidos], ignore_index=True)


,pagina,titulo,conteudo,status
0,1.0,(CO) Consulta Correspondência\CO_0001,COD_TIPO: CO\nNUM_ANTDOC: 1\nPAGINAÇÃO: \nTIPO...,sucesso
1,2.0,(CO) Consulta Correspondência\CO_0002,COD_TIPO: CO\nNUM_ANTDOC: 2\nPAGINAÇÃO: \nTIPO...,sucesso
2,3.0,(CO) Consulta Correspondência\CO_0003,COD_TIPO: CO\nNUM_ANTDOC: 3\nPAGINAÇÃO: \nTIPO...,sucesso
3,4.0,(CO) Consulta Correspondência\CO_0004,COD_TIPO: CO\nNUM_ANTDOC: 4\nPAGINAÇÃO: \nTIPO...,sucesso
4,5.0,(CO) Consulta Correspondência\CO_0005,COD_TIPO: CO\nNUM_ANTDOC: 5\nPAGINAÇÃO: \nTIPO...,sucesso


Arquivo 'dados_consolidados_completo2.csv' salvo com sucesso!


# 1. Importação e Carga dos Dados
Nesta etapa, carregamos o arquivo CSV consolidado. Usamos o parâmetro `keep_default_na=False` e `na_values=['']` para ter um controle melhor sobre o que é considerado vazio.

In [5]:
import pandas as pd
import re

# carregar o dataset
arquivo = 'dados_consolidados_completo2.csv'

# lemos o arquivo
# o encoding='utf-8' é padrão, mas garantimos para evitar erro de acentuação.
df = pd.read_csv(arquivo)

# visualizar a estrutura inicial
print(f"Linhas: {df.shape[0]} | Colunas: {df.shape[1]}")
display(df.head())

Linhas: 5803 | Colunas: 4


,pagina,titulo,conteudo,status
0,1.0,(CO) Consulta Correspondência\CO_0001,COD_TIPO: CO\nNUM_ANTDOC: 1\nPAGINAÇÃO: \nTIPO...,sucesso
1,2.0,(CO) Consulta Correspondência\CO_0002,COD_TIPO: CO\nNUM_ANTDOC: 2\nPAGINAÇÃO: \nTIPO...,sucesso
2,3.0,(CO) Consulta Correspondência\CO_0003,COD_TIPO: CO\nNUM_ANTDOC: 3\nPAGINAÇÃO: \nTIPO...,sucesso
3,4.0,(CO) Consulta Correspondência\CO_0004,COD_TIPO: CO\nNUM_ANTDOC: 4\nPAGINAÇÃO: \nTIPO...,sucesso
4,5.0,(CO) Consulta Correspondência\CO_0005,COD_TIPO: CO\nNUM_ANTDOC: 5\nPAGINAÇÃO: \nTIPO...,sucesso


# 2. Tratamento de Tipos e Limpeza Básica
Aqui fazemos a normalização dos dados existentes:
1. **Página:** Convertemos de Float (1.0) para Inteiro (1), mantendo `<NA>` onde não houver número.
2. **Textos:** Removemos espaços em branco extras no início e fim das strings (trimming).
3. **IDs:** Criamos uma coluna limpa apenas com o código do documento (ex: CO_0001) para facilitar buscas futuras.

In [6]:
# 1. Ajustar a coluna 'pagina' para números inteiros (Int64 aceita nulos)
# Isso remove o ".0" visual (ex: 5.0 vira 5)
df['pagina'] = pd.to_numeric(df['pagina'], errors='coerce').astype('Int64')

# 2. Limpeza de espaços em branco (strip) em todas as colunas de texto
# Isso evita que " Texto" seja diferente de "Texto"
cols_texto = df.select_dtypes(include=['object']).columns
for col in cols_texto:
    df[col] = df[col].astype(str).str.strip()
    # Corrige onde virou string 'nan' ou 'None' para valor nulo real
    df[col] = df[col].replace({'nan': None, 'None': None, '': None})

# extrair um ID limpo do título
# o título atual é longo e vamos criar uma coluna 'id_doc'
# regex pega tudo que vem depois da última barra invertida
df['id_doc'] = df['titulo'].astype(str).str.extract(r'\\([^\\]+)$')

display(df[['id_doc', 'pagina', 'titulo']].head())

,id_doc,pagina,titulo
0,CO_0001,1,(CO) Consulta Correspondência\CO_0001
1,CO_0002,2,(CO) Consulta Correspondência\CO_0002
2,CO_0003,3,(CO) Consulta Correspondência\CO_0003
3,CO_0004,4,(CO) Consulta Correspondência\CO_0004
4,CO_0005,5,(CO) Consulta Correspondência\CO_0005


# 3. Extração de Metadados 
A coluna `conteudo` possui muitas informações aglutinadas (Remetente, Destinatário, Data, Resumo).
Vamos usar **Expressões Regulares (Regex)** para extrair esses dados para novas colunas, **sem apagar** a coluna `conteudo` original.

Isso permitirá que você filtre por data ou remetente depois, o que é impossível com o texto puro.

In [7]:
# função para extrair padrões do texto cru
def extrair_campo(texto, padrao):
    if pd.isna(texto):
        return None
    match = re.search(padrao, texto, re.MULTILINE | re.IGNORECASE)
    return match.group(1).strip() if match else None

# padrões de regex baseados no formato do seu arquivo
padroes = {
    'remetente_extraido': r'REMETENTE:\s*(.*?)(?:\n|$)',
    'destinatario_extraido': r'DESTINATARIO:\s*(.*?)(?:\n|$)',
    'data_extraida': r'DATA_INICIO:\s*(.*?)(?:\n|$)',
    'tipo_doc_extraido': r'TIPO CORRESPONDÊNCIA:\s*(.*?)(?:\n|$)'
}

# aplica a extração para criar novas colunas
print("Extraindo metadados do conteúdo... isso pode levar alguns segundos.")

for nova_coluna, regex in padroes.items():
    df[nova_coluna] = df['conteudo'].apply(lambda x: extrair_campo(str(x), regex))

# visualizar o resultado das novas colunas
colunas_visualizar = ['id_doc'] + list(padroes.keys())
display(df[colunas_visualizar].head())

Extraindo metadados do conteúdo... isso pode levar alguns segundos.


,id_doc,remetente_extraido,destinatario_extraido,data_extraida,tipo_doc_extraido
0,CO_0001,Iris Abbott,Candido Portinari,1940/11/08,Telegrama
1,CO_0002,Júlio Jorge Abeid Filho,Candido Portinari,1956/10/04,Carta
2,CO_0003,Lívio Abramo,Candido Portinari,1945/11/26,Carta
3,CO_0004,Tharcema Cunha de Abreu,Candido Portinari,1946/ /,Carta
4,CO_0005,Aníbal Freire,Júlio Prestes,1926/06/05,Carta


In [8]:
display(df[colunas_visualizar].head(50))

,id_doc,remetente_extraido,destinatario_extraido,data_extraida,tipo_doc_extraido
0,CO_0001,Iris Abbott,Candido Portinari,1940/11/08,Telegrama
1,CO_0002,Júlio Jorge Abeid Filho,Candido Portinari,1956/10/04,Carta
2,CO_0003,Lívio Abramo,Candido Portinari,1945/11/26,Carta
3,CO_0004,Tharcema Cunha de Abreu,Candido Portinari,1946/ /,Carta
4,CO_0005,Aníbal Freire,Júlio Prestes,1926/06/05,Carta
5,CO_0006,Manoel de Abreu,Candido Portinari,1950/08/23,Bilhete
6,CO_0007,Stahlembrecher,Candido Portinari,1928/08/27,Bilhete
7,CO_0008,Olegário Mariano,Baptista Portinari,1928/ /,Telegrama
8,CO_0009,Ronald de Carvalho,Raul Tavares,1929/06/27,Carta
9,CO_0010,Rose Marie Cimonier,Candido Portinari,1930/02/05,Carta


# 4. Análise de Qualidade e Exportação
Verificamos quantas linhas continuam sem conteúdo (para seu controle manual posterior) e salvamos o arquivo tratado.

In [9]:
# contar quantos documentos ainda estão vazios
vazios = df['conteudo'].isna() | (df['conteudo'] == '')
print(f"Total de documentos tratados: {len(df)}")
print(f"Total de documentos com conteúdo vazio (para preenchimento manual): {vazios.sum()}")

# reordenar colunas para deixar as mais importantes no início, mas mantendo TODAS
cols_iniciais = ['id_doc', 'pagina', 'status', 'data_extraida', 'remetente_extraido', 'destinatario_extraido']
# Pega o resto das colunas que não estão na lista de iniciais
cols_restantes = [c for c in df.columns if c not in cols_iniciais]
df_final = df[cols_iniciais + cols_restantes]

'''# salvar
arquivo_saida = 'dados_tratados_final.csv'
df_final.to_csv(arquivo_saida, index=False)

print(f"Arquivo '{arquivo_saida}' gerado com sucesso!")'''

Total de documentos tratados: 5803
Total de documentos com conteúdo vazio (para preenchimento manual): 1672


'# salvar\narquivo_saida = \'dados_tratados_final.csv\'\ndf_final.to_csv(arquivo_saida, index=False)\n\nprint(f"Arquivo \'{arquivo_saida}\' gerado com sucesso!")'

In [10]:
#shape do df_final
print(f"Shape do DataFrame final: {df_final.shape}")

Shape do DataFrame final: (5803, 9)


In [11]:
# Ver quais são as 90 correspondências que estão faltando entre CO_0001 e CO_5892
ids_esperados = {f'CO_{str(i).zfill(4)}' for i in range(1, 5893)}
ids_presentes = set(df_final['id_doc'].dropna().unique())
ids_faltando = ids_esperados - ids_presentes
print(f"Total de correspondências esperadas: {len(ids_esperados)}")
print(f"Total de correspondências presentes: {len(ids_presentes)}")
print(f"Total de correspondências faltando: {len(ids_faltando)}")

# listar as ids faltando
print("IDs faltando:")
for id_faltando in sorted(ids_faltando):
    print(id_faltando)


Total de correspondências esperadas: 5892
Total de correspondências presentes: 5802
Total de correspondências faltando: 91
IDs faltando:
CO_0502
CO_0560
CO_0619
CO_0707
CO_0849
CO_1004
CO_1745
CO_1793
CO_1798
CO_1807
CO_1818
CO_1875
CO_1950
CO_1953
CO_1954
CO_1955
CO_1956
CO_1957
CO_1958
CO_2043
CO_2095
CO_2192
CO_2204
CO_2769
CO_3027
CO_3028
CO_3029
CO_3037
CO_3061
CO_3062
CO_3066
CO_3083
CO_3109
CO_3123
CO_3125
CO_3136
CO_3164
CO_3165
CO_3166
CO_3167
CO_3185
CO_3208
CO_3439
CO_3506
CO_3629
CO_3674
CO_3890
CO_4043
CO_4047
CO_4048
CO_4050
CO_4051
CO_4053
CO_4057
CO_4063
CO_4064
CO_4087
CO_4098
CO_4099
CO_4101
CO_4116
CO_4251
CO_4306
CO_4307
CO_4308
CO_4310
CO_4311
CO_4312
CO_4313
CO_4320
CO_4328
CO_4344
CO_4353
CO_4394
CO_4395
CO_4405
CO_4406
CO_4409
CO_4412
CO_4416
CO_4420
CO_4681
CO_4686
CO_4689
CO_4690
CO_5190
CO_5308
CO_5318
CO_5431
CO_5539
CO_5673


In [12]:
#usando a lista de ids faltando e os dados_extraidos3.csv para ver quais desses ids não existem no site
faltando = pd.read_csv('dados_extraidos3.csv')

#criando coluna de id_doc em faltando
faltando['id_doc'] = faltando['titulo'].astype(str).str.extract(r'\\([^\\]+)$')

#confere se os id_docs são strings
faltando['id_doc'] = faltando['id_doc'].astype(str)

#apaga as linhas que não são CO
faltando = faltando[faltando['id_doc'].str.startswith('CO_')]
#display(faltando['id_doc'])

#cria set de ids extraidos
ids_extraidos = set(faltando['id_doc'].dropna().unique())

#print(ids_extraidos)

ids_faltando_no_site = ids_faltando - ids_extraidos
print(f"Total de IDs faltando que não existem no site: {len(ids_faltando_no_site)}")
print("IDs faltando que não existem no site:")
for id_faltando in sorted(ids_faltando_no_site):
    print(id_faltando)


Total de IDs faltando que não existem no site: 73
IDs faltando que não existem no site:
CO_1807
CO_1875
CO_2043
CO_2095
CO_2192
CO_2769
CO_3027
CO_3028
CO_3029
CO_3037
CO_3061
CO_3062
CO_3066
CO_3083
CO_3109
CO_3123
CO_3125
CO_3136
CO_3164
CO_3165
CO_3166
CO_3167
CO_3185
CO_3208
CO_3439
CO_3506
CO_3629
CO_3674
CO_3890
CO_4043
CO_4047
CO_4048
CO_4050
CO_4051
CO_4053
CO_4057
CO_4063
CO_4064
CO_4087
CO_4098
CO_4099
CO_4101
CO_4116
CO_4251
CO_4306
CO_4307
CO_4308
CO_4310
CO_4311
CO_4312
CO_4313
CO_4320
CO_4328
CO_4344
CO_4353
CO_4394
CO_4395
CO_4405
CO_4406
CO_4409
CO_4412
CO_4416
CO_4420
CO_4681
CO_4686
CO_4689
CO_4690
CO_5190
CO_5308
CO_5318
CO_5431
CO_5539
CO_5673


Descobrir as páginas específicas que cada uma dessas correspondências faltantes para depois completar o df

In [13]:
# carregar os arquivos
# 'on_bad_lines' para pular linhas com erro se houver, e encoding utf-8
df_docvirt = pd.read_csv('dados_consolidados_completo2.csv')
df_extraidos = pd.read_csv('dados_extraidos3.csv')

# ajustar 'dados_extraidos' 
# o arquivo extraído tem muitas colunas vazias no final, vamos pegar apenas as úteis
cols_uteis = ['titulo', 'conteudo']
df_extraidos = df_extraidos[cols_uteis].copy()

# adicionar as colunas que faltam para ficar no padrão do docvirt
# se não houver informação, deixamos como None (NaN) ou um texto padrão
df_extraidos['pagina'] = None 
df_extraidos['status'] = 'extraido_novo' # Marcador para você saber a origem depois

# reordenar as colunas do extraído para ficarem na mesma ordem do docvirt
colunas_ordem = df_docvirt.columns.tolist()
# garante que o df_extraidos tenha todas as colunas necessárias
for col in colunas_ordem:
    if col not in df_extraidos.columns:
        df_extraidos[col] = None
        
df_extraidos = df_extraidos[colunas_ordem]

# juntar e completar
# concatenamos os dois e colocamos o docvirt primeiro, pois ele parece mais completo (tem paginação)
df_final = pd.concat([df_docvirt, df_extraidos], ignore_index=True)

# remover duplicatas
# se o mesmo 'titulo' existir nos dois, mantemos o primeiro (que veio do docvirt)
# isso garante que completamos a lista sem criar linhas repetidas
df_final = df_final.drop_duplicates(subset=['titulo'], keep='first')

# visualziar e salvar
print(f"Total de linhas no DocVirt original: {len(df_docvirt)}")
print(f"Total de linhas no Extraídos original: {len(df_extraidos)}")
print(f"Total de linhas após juntar e limpar: {len(df_final)}")

# exibir as primeiras linhas 
display(df_final.head())

# salva
df_final.to_csv('dados_consolidados_completo3.csv', index=False)
print("Arquivo 'dados_consolidados_completo3.csv' salvo com sucesso!")

Total de linhas no DocVirt original: 5803
Total de linhas no Extraídos original: 92
Total de linhas após juntar e limpar: 5836


/tmp/ipykernel_36144/972407154.py:27: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_final = pd.concat([df_docvirt, df_extraidos], ignore_index=True)


,pagina,titulo,conteudo,status
0,1.0,(CO) Consulta Correspondência\CO_0001,COD_TIPO: CO\nNUM_ANTDOC: 1\nPAGINAÇÃO: \nTIPO...,sucesso
1,2.0,(CO) Consulta Correspondência\CO_0002,COD_TIPO: CO\nNUM_ANTDOC: 2\nPAGINAÇÃO: \nTIPO...,sucesso
2,3.0,(CO) Consulta Correspondência\CO_0003,COD_TIPO: CO\nNUM_ANTDOC: 3\nPAGINAÇÃO: \nTIPO...,sucesso
3,4.0,(CO) Consulta Correspondência\CO_0004,COD_TIPO: CO\nNUM_ANTDOC: 4\nPAGINAÇÃO: \nTIPO...,sucesso
4,5.0,(CO) Consulta Correspondência\CO_0005,COD_TIPO: CO\nNUM_ANTDOC: 5\nPAGINAÇÃO: \nTIPO...,sucesso


Arquivo 'dados_consolidados_completo3.csv' salvo com sucesso!


In [14]:
df_parcial = pd.read_csv('dados_consolidados_completo3.csv')
# quantidade de linhas com 'conteudo' vazio ou nulo
n_vazios = df_parcial['conteudo'].isna().sum() + (df_parcial['conteudo'] == '').sum()
print(n_vazios)

# Contando quantos COD_TIPO: AP existem
n_ap = df_parcial['conteudo'].fillna('').str.startswith('COD_TIPO: AP').sum()
print(n_ap)

1672
15


In [15]:
# Filtrando os códigos para ter só os CO_NNNNN e tirar os AP_NN.N
# Também filtrando os vazios
df_final = df_final[df_final['conteudo'].fillna('').str.startswith('COD_TIPO: CO')]
print(df_final.shape)

#Conferência de quantidade
print(df_final.shape[0] + 1672 + 91 - 15)

(4147, 4)
5895


In [16]:
# Vendo se df_final tem algum CO_ duplicado
print(f"Total de CO_ únicos: {df_final['titulo'].nunique()}")

# mais verificações diversas
print(f"Total de linhas no df_final: {len(df_final)}")

# Printa todos os tipos de status
print("Status únicos no df_final:", df_final['status'].unique())

#ordena o df por CO_NNNN crescentemente
df_final['id_doc'] = df_final['titulo'].astype(str).str.extract(r'\\([^\\]+)$')
df_final = df_final.sort_values(by='id_doc')

#transforma a coluna 'pagina' em inteiro, usando pd.to_numeric para lidar com possíveis erros e convertendo para Int64 para aceitar nulos
df_final['pagina'] = pd.to_numeric(df_final['pagina'], errors='coerce').astype('Int64')

# Completa a coluna página, substituindo os <NA> pelo valor correto ordenado, de acordo com a posição da linha no df
df_final['pagina'] = df_final['pagina'].ffill()

#teste com o CO_0502, que estava com página vazia
display(df_final[df_final['id_doc'] == 'CO_0502'])

Total de CO_ únicos: 4147
Total de linhas no df_final: 4147
Status únicos no df_final: ['sucesso' 'extraido_novo']


,pagina,titulo,conteudo,status,id_doc
5805,803,(CO) Consulta Correspondência\CO_0502,COD_TIPO: CO\nNUM_ANTDOC: 502\nPAGINAÇÃO: \nTI...,extraido_novo,CO_0502


In [17]:
# Salvar como Dados Finais
df_final.to_csv('dados_finais_temp.csv', index=False)